<h1><center>Examen Inteligencia Artificial</center></h1>


### Nombre:
Andrés Zeas G.

## Objetivo

Consolidar los conocimientos adquiridos en clase para los métodos de búsqueda y bases de datos orientadas a grafos.

## Enunciado

Diseñe y desarrolle un sistema recopilador que permita obtener las noticias o facebook de los candidatos y almacenar dentro de una base de datos orientados a grafos:

Webscraping es la técnica de extraer datos contenidos en un formato no estructurado en una página web y llevarlos a una estructura fácil de usar.
Es por ello, que se desea crear nuevos métodos que permitan la recopilación masiva de informaciones para su posterior estudio y correlación en forma de big data.
En base a ello, vamos a obtener los datos de lo que esta hablando las noticias de los candidatos dentro del Ecuador y almacenar los datos dentro de una base de datos orientadas a grafos.
https://www.google.com/search?tbm=nws&sxsrf=ALeKk01Q48xyd7421N_rrjzDN56Mnp8FTg:1607467777701&q=candidatos+ecuador&spell=1&sa=X&ved=0ahUKEwioo4KPvL_tAhXFrFkKHXv8CC8QBQgvKAA&biw=1920&bih=915&dpr=1


* Generar un modelo que permita obtener y almacenar los datos en los grafos.
* Vincular los datos con el candidato seleccionado.
* Se debe tener al menos 1000 nodos generados, en caso de tener estos datos seleccionar al partido (UNES) (UNES AZUAY) (Candidato Presidencial)
* Obtener de la noticia: el Link, mensaje, fecha
* Facebook: Comentarios, Publicaciones, Amigos, Likes, Seguidores, etc.
* No se debe repetir los candidatos a asambleístas dentro del Foro del partido político de UNES AZUAY (10 primeros) .
* Se puede utilizar cualquier herramienta o procesamiento para el WebScarping.
* Generar sus análisis, conclusiones y recomendaciones en base a  los datos


## Neo4j + Python
Para instalar la libreria solo tenemos que ejecutar

`pip install neo4j`

Procedemos a verificar que la instalacion este correcta con la siguiente instrucción.

## Ejecucion desde Python
A continuacion configuran la cadena de conexion y verifican que este correctamente instalado la libreria si genera un error de seguridad TTL [4] agregar 'encrypted=False'

In [88]:
from neo4j import GraphDatabase

## Desarrollo

Haremos uso de Beautiful Soup la cual es una biblioteca de Python que utiliza su analizador html / xml preinstalado y convierte la página web / html / xml en un árbol que consta de etiquetas, elementos, atributos y valores. Para ser más exactos, el árbol consta de cuatro tipos de objetos, Tag, NavigableString, BeautifulSoup y Comment. Este árbol se puede "consultar" utilizando los métodos / propiedades del objeto BeautifulSoup que se crea a partir de la biblioteca del analizador.

Procedemos a instalar Beautiful Soup con el siguiente comando: `pip install beautifulsoup4` y lo importamos a nuestro proyecto.

In [89]:
from bs4 import BeautifulSoup
import requests

Generamos el diseño de como van a estar conformados nuestros nodos dentro de la base de Neo4j

In [90]:
class Neo4jService(object):
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
        
    def close(self):
        self._driver.close()      
    
    #Creación de Nodos
    def nodo_cabeceraTitulo(self, tx, nombre):
        tx.run("CREATE (:Titulo {nombre: $nombre})", nombre=nombre)
        
    def nodo_candidatoAsambleista(self, tx, nombre):
        tx.run("CREATE (:Candidato {nombre: $nombre})", nombre=nombre)
        
    def nodo_noticia(self, tx, nombre):
        tx.run("CREATE (:Noticias {nombre: $nombre})", nombre=nombre)
        
    def nodo_fechaNoticia(self, tx, nombre):
        tx.run("CREATE (:Fecha {nombre: $nombre})", nombre=nombre)   
        
    def nodo_contenidoNoticia(self, tx, nombre):
        tx.run("CREATE (:Contenido {nombre: $nombre})", nombre=nombre)   
        
    def nodo_titulosNoticias(self, tx, nombre):
        tx.run("CREATE (:Titulos {nombre: $nombre})", nombre=nombre)
        
    def nodo_fechasNoticias(self, tx, nombre):
        tx.run("CREATE (:Fechas {nombre: $nombre})", nombre=nombre)
     
    #Relaciones
    def relacion_noticia(self, tx, nombre_noticias, nombre_noticia):
        tx.run("MATCH (a:Noticias {nombre: $nombre_noticias}) "
               "MATCH (b:Contenido {nombre: $nombre_noticia}) "
               "MERGE (a)-[:Noticias]->(b)",
               nombre_noticias=nombre_noticias, nombre_noticia=nombre_noticia)
        
    def relacion_titulosCabeceras(self, tx, nombre_titulos, nombre_titulo):
        tx.run("MATCH (a:Titulo {nombre: $nombre_titulos}) "
               "MATCH (b:Titulos {nombre: $nombre_titulo}) "
               "MERGE (a)-[:TituloNoticia]->(b)",
               nombre_titulos=nombre_titulos, nombre_titulo=nombre_titulo)
        
    def relacion_fechaNoticias(self, tx, tiempo_fechas, nombre_fecha):
        tx.run("MATCH (a:Fecha {nombre: $tiempo_fechas}) "
               "MATCH (b:Fechas {nombre: $nombre_fecha}) "
               "MERGE (a)-[:Fecha]->(b)",
               tiempo_fechas=tiempo_fechas, nombre_fecha=nombre_fecha)

    def relacion_candidato_tituloNoticia(self, tx, candidato, titulo_noticia):
        tx.run("MATCH (a:Candidato {nombre: $candidato}) "
               "MATCH (b:Titulo {nombre: $titulo_noticia}) "
               "MERGE (a)-[:Titulos]->(b)",
               candidato=candidato, titulo_noticia=titulo_noticia)
        
    def relacion_candidato_fechaNoticia(self, tx, candidato, fecha_noticia):
        tx.run("MATCH (a:Candidato {nombre: $candidato}) "
               "MATCH (b:Fecha {nombre: $fecha_noticia}) "
               "MERGE (a)-[:Fechas]->(b)",
               candidato=candidato, fecha_noticia=fecha_noticia)
    
    def relacion_candidato_contenidoNoticia(self, tx, candidato, noticia_contenido):
        tx.run("MATCH (a:Candidato {nombre: $candidato}) "
               "MATCH (b:Noticias {nombre: $noticia_contenido}) "
               "MERGE (a)-[:Contenido]->(b)",
               candidato=candidato, noticia_contenido=noticia_contenido)

Se procede a la conección con la base de datos.
Manda a crear los nodos.

In [91]:
neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'admin')
with neo4j._driver.session() as session:
    session.write_transaction(neo4j.nodo_candidatoAsambleista , "Candidato")
    session.write_transaction(neo4j.nodo_cabeceraTitulo , "Titulos")
    session.write_transaction(neo4j.nodo_fechaNoticia , "Fecha")
    session.write_transaction(neo4j.nodo_noticia , "Noticias")

#Enlaces de datos
with open("datos.txt","r") as archivo:
    for enlace in archivo:
        print('ENLACE')
        print(enlace)
        pageGoo= requests.get(enlace)
        soupGoo = BeautifulSoup(pageGoo.content,'html.parser')
        resultaFecha = soupGoo.find_all("span",{"class":"r0bn4c rQMQod"})
        resultaTitulo = soupGoo.find_all("div",{"class":"BNeawe vvjwJb AP7Wnd"})
        resultaTexto = soupGoo.find_all("div",{"class":"kCrYT"}) 
        contenidoTextoGoogle=list()
        
        for resu in resultaTexto:
            contenidoTextoGoogle.append(resu.text)
        auxGo=list(set(contenidoTextoGoogle)) 
        
        for ll in auxGo:
            session.write_transaction(neo4j.nodo_contenidoNoticia,ll)
            session.write_transaction(neo4j.relacion_noticia,"Noticias",ll)
        tituloGoo=list()
        
        for i in resultaTitulo:
            tituloGoo.append(i.text) 
            
        for titulo in tituloGoo:
            session.write_transaction(neo4j.nodo_titulosNoticias,titulo)
            session.write_transaction(neo4j.relacion_titulosCabeceras,"Titulos",titulo)  
            print('TITULO')
            print(titulo)
        enlaceFecha=list()
        
        for e2 in resultaFecha:
            enlaceFecha.append(e2.text)
            
        for fecha in enlaceFecha:
            print('FECHA')
            print(fecha)
            session.write_transaction(neo4j.nodo_fechasNoticias,fecha)
            session.write_transaction(neo4j.relacion_fechaNoticias,"Fecha",fecha) 
            
session.write_transaction(neo4j.relacion_candidato_tituloNoticia,"Candidato","Titulos")
session.write_transaction(neo4j.relacion_candidato_contenidoNoticia,"Candidato","Noticias")
session.write_transaction(neo4j.relacion_candidato_fechaNoticia,"Candidato", "Fecha")
print("Todos los nodos han sido generados!!")

ENLACE
https://www.google.com/search?tbm=nws&sxsrf=ALeKk00QWMy18iOxpJ5tiBl3PJ8svZj0dg%3A1608624449508&ei=QanhX8HHHqSu5wKDkZDQBQ&q=Andres+Arauz&oq=Andres+Arauz&gs_l=psy-ab.3...147647.147647.0.147974.1.1.0.0.0.0.165.165.0j1.1.0....0...1c.1.64.psy-ab..0.0.0....0._PBXLrrbgLo

TITULO
Candidato presidencial Andrés Arauz contagiado de Covid-19
TITULO
El candidato presidencial Andrés Arauz informa que dio positivo a ...
TITULO
Candidato presidencial Andrés Arauz da positivo para COVID-19
TITULO
Andrés Arauz, el candidato del correísmo, tiene COVID-19
TITULO
Andrés Arauz, el economista que quiere emular a Rafael Correa
TITULO
Andrés Arauz pide vigilancia internacional tras retraso en ...
TITULO
La dupla presidencial de Andrés Arauz y Carlos Rabascall, oficialmente calificada para participar en los comicios nacionales
TITULO
Andrés Arauz: Presentamos un programa de apoyo de emergencia ...
TITULO
Desechan nuevo recurso contencioso electoral, en contra de binomio presidencial Arauz-Rabascall
TITUL

TITULO
Arauz promete entregar mil dólares a familias ecuatorianas en su ...
TITULO
Arauz anuncia que su gobierno fortalecerá la dolarización con la ...
TITULO
Dirigente de Pachakutik plantea la expulsión de sus filas de la ...
TITULO
Gana Arauz: bienvenida Venezuela : Opinión
FECHA
Hace 1 semana
FECHA
 · 
FECHA
Hace 1 semana
FECHA
 · 
FECHA
hace 1 día
FECHA
 · 
FECHA
hace 1 mes
FECHA
 · 
FECHA
hace 16 horas
FECHA
 · 
FECHA
Hace 3 semanas
FECHA
 · 
FECHA
Hace 1 semana
FECHA
 · 
FECHA
Hace 3 semanas
FECHA
 · 
FECHA
hace 4 días
FECHA
 · 
FECHA
Hace 3 semanas
FECHA
 · 
ENLACE
https://www.google.com/search?q=Andres+Arauz&tbm=nws&sxsrf=ALeKk03FbZTpX_ZEoVdySdoh3nUGAjmuvA:1608624714752&ei=SqrhX7ylLaWx5wLMhaz4AQ&start=70&sa=N&ved=0ahUKEwi88suFkuHtAhWl2FkKHcwCCx84PBDy0wMIkQE&biw=1366&bih=625&dpr=1

TITULO
PLACA CONTRA CORREA EN PALACIO DE CARONDELET
TITULO
El binomio correísta acude a la ONU por la inscripción de su ...
TITULO
Lasso, 23%; Arauz, 13%; Pérez, 10.7%, según CEDATOS
TITULO
Lasso, sin

FECHA
Hace 1 semana
FECHA
 · 
FECHA
hace 18 horas
FECHA
 · 
FECHA
Hace 2 semanas
FECHA
 · 
FECHA
Hace 4 semanas
FECHA
 · 
FECHA
hace 17 horas
FECHA
 · 
FECHA
Hace 1 semana
FECHA
 · 
ENLACE
https://www.google.com/search?q=Andres+Arauz+candidato+presidencial&tbm=nws&sxsrf=ALeKk02uR1fw6DKR7A2Xjsb7TjQSWU0S9A:1608650258828&ei=Eg7iX_aJMsqD5wK-vKGIDA&start=30&sa=N&ved=0ahUKEwj22vqZ8eHtAhXKwVkKHT5eCME4FBDy0wMIigE&biw=1366&bih=625&dpr=1

TITULO
FALSO: Arauz tiene 3 juicios pendientes por malversación de fondos y por un accidente de tránsito. Además, los títulos de 3er y 4to nivel son falsos
TITULO
Corte Constitucional admitió a trámite denuncia presentada por ...
TITULO
Lasso dice que el plan de gobierno de Arauz se prepara en ...
TITULO
Binomio correísta pide vigilancia internacional tras retraso en aprobación de candidatura
TITULO
Candidatura del correísmo sigue sin resolución
TITULO
Unión por la Esperanza de Ecuador presentará plan económico
TITULO
Contencioso da trámite a nueva impugnación 

TITULO
Izquierda latinoamericana recobra impulso y espera su nueva ola
TITULO
Andrés Arauz se reunirá en Bolivia con Arce, Iglesias y Morales
TITULO
Sputnik V es recibida con ambigüedad por Occidente, a pesar de ...
TITULO
Las propuestas con las que Andrés Arauz planea llegar a Carondelet
TITULO
Carlos Rabascall, el nuevo binomio de Andrés Arauz que dejó los medios públicos en 2017
TITULO
Centro Democrático inició sus primarias entre dos opciones: Andrés ...
TITULO
Experto: "Las políticas neoliberales de Lenín Moreno han exacerbado las condiciones en la pandemia"
TITULO
Andrés Arauz prioriza las necesidades del pueblo ecuatoriano
TITULO
Andrés Arauz pide a Naciones Unidas y a la UE designar Comisión para vigilar de cerca labor del CNE
TITULO
TCE acepta a trámite recurso del movimiento Ahora en contra del ...
FECHA
hace 19 horas
FECHA
 · 
FECHA
hace 1 mes
FECHA
 · 
FECHA
hace 1 día
FECHA
 · 
FECHA
hace 1 mes
FECHA
 · 
FECHA
hace 3 meses
FECHA
 · 
FECHA
hace 4 meses
FECHA
 · 
FECHA
Hace 

## Conclusiones y recomendaciones

* Investigar el uso de las diferentes herramientas que permiten web scraping de acuerdo al tipo de información que se requiera obtener.
* Este método de etracción de  contenido nos resulta bastante util para realizar analisis de datos a grandes escalas.
* Las bases de datos Neo4j debido a su versatilidad nos permite manipular la información a nuestra conveniencia para relizar diferentes tipso de analisis, asi como los algoritmos predefinidos que nos permiten un analisis más profundo de la información.

## Bibliografía y referencias

* Beaufiful Soap
https://riptutorial.com/es/beautifulsoup#:~:text=Beautiful%20Soup%20es%20una%20biblioteca,%2C%20NavigableString%2C%20BeautifulSoup%20y%20Comment.

* Neo4j + Python
https://neo4j.com/developer/python/

